In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [12]:
# Проверяем доступна ди GPU
!nvidia-smi  # Должна появиться таблица с информацией о GPU

In [ ]:
# Проверяем доступна ди GPU
import torch
print(torch.cuda.is_available())

In [ ]:
import pandas as pd
import time
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

# 1. Регистрация COCO-val
dataset_name = "coco_val"
json_ann = "annotations/instances_val2017.json"
image_root = "val2017/"
register_coco_instances(dataset_name, {}, json_ann, image_root)

# 2. Настройка Detectron2
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
predictor = DefaultPredictor(cfg)

# 3. Создание evaluator и loader
evaluator = COCOEvaluator(dataset_name, cfg, False, output_dir="./coco_eval")
val_loader = build_detection_test_loader(cfg, dataset_name)

# 4. Измерение времени инференса и сбор метрик
num_images = len(DatasetCatalog.get(dataset_name))
start_time = time.time()
metrics_dict = inference_on_dataset(predictor.model, val_loader, evaluator)
elapsed = time.time() - start_time
fps = num_images / elapsed

# 5. Выбор нужных метрик
bbox = metrics_dict["bbox"]
results = {
    "mAP@[.50:.95]": bbox["AP"],
    "mAP@.50": bbox["AP50"],
    "mAP@.75": bbox["AP75"],
    "AP@.50": bbox["AP50"],
    "AP@.75": bbox["AP75"],
    "AR@100": bbox.get("AR@100", None),
    "Precision≈AP50": bbox["AP50"],
    "Recall≈AR@100": bbox.get("AR@100", None),
    "FPS": round(fps, 2)
}

# 6. Печать и таблица
df = pd.DataFrame([results])
print(df.to_markdown(index=False))